# How to (actually) make games with python
## Slides at [code.achi.se/slides](https://github.com/achifaifa/slides)

### Yuri Numerov (Achifaifa)
### FOSDEM'16
#### 2016-01-30

### [achi.se](http://achi.se) / [@Achifaifa](https://twitter.com/achifaifa) / [achi (a) hush.ai](http://argh.wargh)

## About me

* ¯\\_(ツ)_/¯

### My issue with `""gamedev""` talks

* Mostly people selling frameworks

* No game talks without pygame! :<

* Very rarely about actually making games

### So, let's fix that!

We'll be making not one, but two games (TWO!)

### Rules:

* No pygame

* No custom frameworks

* No dependencies

* Just whatever you have in the standard library!

### But first, some basics

#### Q: What is a game?

#### A: A world (with rules) we can manipulate (With or without winning/losing conditions)

#### Q: What is the goal of a game?

#### A:  ¯\\_(ツ)_/¯

### Round 1

Clue: The only winning move is not to play

![](./img/ttt.png)

### What are the rules of the game?

* Played in a 3*3 grid

* Two players

* Each player draws a symbol in an empty position of the grid

* First player to get three symbols in a row wins

### First, we import some stuff

In [7]:
#! usr/bin/env python
     
from os import system as DO

### Then we initialize the world and the players

In [1]:
board=[[" " for i in range(3)] for i in range(3)]
players=["X","O"]

Remember, the in-game players are the tools you give the actual player to interact with the world

### And we enter the main loop

Normally an infinite loop, but we can use a `for` here :)

In [2]:
for i in range(9):
    pass

### Think about what you do in the loop:
That's a turn, a cycle, or the minimum number of actions you need to repeat to play

* Clear the screen (Begin the cycle)

In [14]:
DO('clear')

* Print the board (Let the player/s know the status of the game)

In [13]:
print "\n-----\n".join(["|".join(j) for j in board])

* Ask for a move to a player (Get player feedback)

In [15]:
x,y=[int(k)-1 for k in raw_input("Player %s >"%players[i%2]).split(',')]

* Fill the cell (Modify the world based on that feedback and the rules of the game)

In [16]:
board[y][x]=players[i%2] if board[y][x]==" " else board[y][x]

* Check if anyone has won (If the status of the world matches a winning/losing conditions)

In [2]:
# Boring! The player can do that

### And that's it!

In [ ]:
#! usr/bin/env python

from os import system as DO

board=[[" " for i in range(3)] for i in range(3)]
players=["X","O"]

for i in range(9):
    DO('clear')
    print "\n-----\n".join(["|".join(j) for j in board])
    x,y=[int(k)-1 for k in raw_input("Player %s >"%players[i%2]).split(',')]
    board[y][x]=players[i%2] if board[y][x]==" " else board[y][x]

###                                                      Demo?
# 🔥😱🔧

### What did we learn?

* Need to define rules, parameters and conditions

* Preferably before we start coding

* Need to create a world, a human-readable representation of the world and a (consistent) way to interact with it

* Need to think about the loop

 * Give feedback to player
 * Receive his choice of interaction with the world
 * Process the interaction
 * Check the world compare against conditions

* In all games, lots of code will be there just to prevent the player from breaking things

### Round 2
Let's make something slightly more complex

[tetrees image]

### The challenges (I)

* The principles are the same

* World changes even without user interaction

* Need to capture non-blocking input

### StackOverflow copypaste that works:

In [3]:
def pressed():

  def isData():
    return select.select([sys.stdin], [], [], 0.01)==([sys.stdin], [], [])

  c=""
  old_settings=termios.tcgetattr(sys.stdin)
  try:
    tty.setcbreak(sys.stdin.fileno())
    if isData():
      c=sys.stdin.read(1)
  finally:
    termios.tcsetattr(sys.stdin, termios.TCSADRAIN, old_settings)
    return c

### The challenges (II)

* The speed must be independent from user interaction

* Also it has to be independent from computer speed or main loop cycles

* Need to update at a constant rate

### The solution

* We capture the input whenever the player presses a key

* We store it somewhere

* And only update the screen based on a real time counter

In [22]:
def loopmanage():

  timem["timepool"]=0 if timem["timepool"]*1000>=300 else timem["timepool"]+time.time()-timem["previoustime"]
  timem["previoustime"]=time.time()
  return not timem["timepool"]

### Now, rinse and repeat :)

### We import what we need...

In [2]:
#! /usr/bin/env python

import copy, os, random, select, sys, termios, time, tty

### Define our world...

In [4]:
world=[["." for i in range(10)] for i in range(22)]
pieces=[["#","#","#","#"],
        [["#","#","."],[".","#","#"]],
        [[".","#","#"],["#","#","."]],
        [["#",".","."],["#","#","#"]],
        [[".",".","#"],["#","#","#"]],
        [[".","#","."],["#","#","#"]],
        [["#","#"],["#","#"]]]

### Then the tools the player has to interact with it...
(And some extras)

In [5]:
keys=["c", "z", "v", "x", "q"]
timem={"timepool":0,"previoustime":0}
curpiece={"piece":copy.copy(random.choice(pieces)), "coords":[0,3]}
score={1:0, 2:0, 3:0, 4:0, "drops":0}

### And create the loop

In [10]:
def mainloop():

  tempkey=pressed()
  try: lastkey=tempkey if tempkey in keys else lastkey
  except: lastkey=""
  if lastkey=="q": gameover()

  if loopmanage() or lastkey:

    movepiece(lastkey)
    output()

if __name__=="__main__":
  while 1:
    mainloop()

### And once in the loop, we move the piece...

In [13]:
def movepiece(direction):

  if direction==keys[0] or not timem["timepool"]: 
    if curpiece["coords"][0]+len(curpiece["piece"])>=22 or collision("down"):
      merge()
    else:
      curpiece["coords"][0]+=1
      if direction==keys[0]: score["drops"]+=1

  if direction==keys[1] and curpiece["coords"][1]-1>=0 and not collision("left"): 
    curpiece["coords"][1]-=1

  if direction==keys[2] and curpiece["coords"][1]+1+len(curpiece["piece"][0])<=10 and not collision("right"): 
    curpiece["coords"][1]+=1

  if direction==keys[3]: curpiece["piece"]=rotate()

### (Detecting collisions) (Boring part)

In [14]:
def collision(direction):

  piece=curpiece["piece"]
  coords=curpiece["coords"]

  if direction=="down":
    for numi, i in enumerate(piece[-1]):
      try:
        if i=="#" and world[coords[0]+len(piece)][coords[1]+numi]=="#": return 1
        if piece[-2][numi]=="#" and world[coords[0]+len(piece)-1][coords[1]+numi]=="#": return 1
        if piece[-3][numi]=="#" and world[coords[0]+len(piece)-2][coords[1]+numi]=="#": return 1
      except IndexError: pass

  elif direction=="left":
    for numi, i in enumerate(piece):
      try:
        if i[0]=="#" and world[coords[0]+numi][coords[1]-1]=="#": return 1
        if i[1]=="#" and world[coords[0]+numi][coords[1]]=="#": return 1
        if i[2]=="#" and world[coords[0]+numi][coords[1]+1]=="#": return 1
      except IndexError: pass

  elif direction=="right":
    for numi, i in enumerate(piece):
      try:
        if i[-1]=="#" and world[coords[0]+numi][coords[1]+len(piece[0])]=="#": return 1
        if i[-2]=="#" and world[coords[0]+numi][coords[1]+len(piece[0])-1]=="#": return 1
        if i[-3]=="#" and world[coords[0]+numi][coords[1]+len(piece[0])-2]=="#": return 1
      except IndexError: pass

  return 0

### ...And merging the piece with the world once it can't go further...

In [24]:
def merge():

  global world    #Shhhh...
  global curpiece #No pain now

  for numi, i in enumerate(curpiece["piece"]):
    for numj, j in enumerate(i):
      if j=="#":
        world[numi+curpiece["coords"][0]][numj+curpiece["coords"][1]]="#"
  curpiece={"piece":copy.copy(random.choice(pieces)), "coords":[0,3]}
  if collision("down"): gameover()

  processlines()

### ...Processing the world to remove lines later.

In [16]:
def processlines():

  global world

  world=[i for i in world if not all(j=="#" for j in i)]
  removed=22-len(world)
  world=[["." for i in range(10)] for i in range(removed)]+world
  if removed: score[removed]+=1

### Oh, and rotating the piece when the proper key is pressed

In [17]:
def rotate():

  temp=[[] for i in curpiece["piece"][0]]
  for i in curpiece["piece"]:
    for numj,j in enumerate(i):
      temp[numj].append(j)
  temp=[i[::-1] for i in temp]
  return temp

### And ending the game if the new piece is already in contact with something

In [18]:
def gameover():

  os.system('clear')
  print "GAME OVER"
  print "\n1: %ix10: %i"%(score[1],score[1]*10)
  print "2: %ix50: %i"%(score[2],score[2]*50)
  print "3: %ix500: %i"%(score[3],score[3]*500)
  print "tetreeses: %ix1337: %i"%(score[4],score[4]*1337)
  print "drops: %i"%(score["drops"])
  print "\nTotal Score: %i"%(score["drops"]+score[1]*10+score[2]*50+score[3]*500+score[4]*1337)
  exit()

### And after that, we show the player the new status of the world

In [21]:
def output():

  os.system('clear')
  tempworld=copy.deepcopy(world)

  for numi, i in enumerate(curpiece["piece"]):
    for numj, j in enumerate(i):
      if j=="#":
        tempworld[numi+curpiece["coords"][0]][numj+curpiece["coords"][1]]=j 

  for i in tempworld: print " ".join(i)
  print "score: %i"%(score["drops"]*2+score[1]*10+score[2]*50+score[3]*500+score[4]*1337)

### ...And repeat until the game is over or the player quits

# Demo?

# 🔥🔥🔥😱 
# 🔧🔥😱😱
# 🔧🔧🔧😱


### Extras (Workshop/Dojo/etc)

### Do not limit yourself to cloning games. Add your own rules!

* Implement levels and speed/score increases

* Make it multiplayer! (Each player -> 1 piece)

* Add a second plane. Make it 3D!


* Add weird pieces with special skills

* Create a new game mode

* Give extra points for removing all the lines

* Add power-ups and special moves

### Other games you can play with

* Moon lander

* Space Invaders

* Mario bros

* Asteroids

* Snake

* Any board game you like

* Other console games (Nethack, Gearhead, DF)

* Rythm games

# How to (actually) make games with python

# Hope you learned something! :D (Q/A)

### Yuri Numerov (Achifaifa)
### FOSDEM'16

### [achi.se](http://achi.se) / [@Achifaifa](https://twitter.com/achifaifa) / [achi (a) hush.ai](http://argh.wargh)